In [84]:
"""
Script converts all mseed files into numpy array. Each array is detrended by fitting a linear function and subtracting that line to remove the DC offset.

"""

'\nScript converts all mseed files into numpy array. Each array is detrended by fitting a linear function and subtracting that line to remove the DC offset.\n\n'

In [22]:
import obspy

import numpy as np
import pandas as pd
from glob import glob
import os
from os.path import join, basename, dirname, exists
import re

In [74]:
import matplotlib.pyplot as plt
from datetime import datetime

In [13]:
ac_calib = 8.2928e-05
name_dic = {'be4':'lower','a3m':'upper','ad8':'horizontal'}

In [20]:
local_data_dir = '/home/zacharykeskinen/Documents/infrasound/data/banner/infrasound/processed'
target_dir = '/home/zacharykeskinen/Documents/infrasound/array_data'
assert exists(target_dir)

In [82]:
fps = glob(join(local_data_dir, '*'))
fps = [f for f in fps if '.1.' not in f]
array_names = np.unique([basename(f)[2:5] for f in fps])
assert set(array_names) == set(name_dic.keys()), 'Missing an array in dict?'
for ext, desc in name_dic.items():
    os.makedirs(join(target_dir, f'{ext}-{desc}'), exist_ok= True)
    array_fps = [f for f in fps if ext in f]
    res = pd.DataFrame()
    if ext != 'ad8':
        for i, fp in enumerate(array_fps):
            if i < 3:
                tr = obspy.read(fp)[0]
                tr.detrend("linear")
                arr = np.array(tr.data * ac_calib)
                t = [datetime.fromtimestamp(t) for t in tr.times("timestamp")]
                stats = tr.stats
                res = pd.concat([res, pd.DataFrame(arr, index = t)])

In [ ]:
in_dir = '/home/zacharykeskinen/data/infrasound/infrasound/processed' #infrasound/banner/infrasound/processed/'
in_dir = '../../data/banner/infrasound/processed'
exists(in_dir)

In [ ]:
# l = os.listdir(in_dir)
l = glob(os.path.join(in_dir, 'c0be4*'))
r = []
for i in l:
    i = os.path.basename(i)
    j = i[5:11]
    if j not in r:
        r.append(j)
r.sort()

In [ ]:
name_dic = {'be4':'lower','a3m':'upper','ad8':'failed array'}
height_dic = {'lower-p0':0.33,'lower-p1':0.66,'lower-p2':1,'upper-p0':1.33,'upper-p1':np.nan,'upper-p2':2}
full_res = {}
for i, day in tqdm(enumerate(r), desc = 'days'):
    # if i == 65:
        ls = glob(join(in_dir, '*'+day+'*'))
        # print(ls)
        res = {}
        for file in ls:
                name = name_dic[basename(file).replace(day,'')[2:5]]
                if name != 'failed array':
                    tr = read(file)[0]
                    stats = tr.stats
                    sps = stats['sampling_rate']
                    start = stats['starttime']
                    end = stats['endtime']
                    channel = stats['channel']
                    name_channel = f'{name}-{channel}'
                    height = height_dic[name_channel]
                    if not np.isnan(height):
                        arr = tr.data
                        arr = arr * ac_calib
                        arr = arr - np.nanmean(arr)
                        # power = arr**2
                        # power = 1/len(power) * np.cumsum(power)[-1]
                        # res[height] = power
                        ARR = fft(arr)
                        f = fftfreq(arr.size, 1/sps)
                        start = int(f1/100*len(ARR))
                        end = int(f2/100*len(ARR))
                        sub_ARR = ARR[start:end]
                        mag = np.abs(sub_ARR**2)
                        power = sum(mag)/len(arr)
                        res[height] = 2* power/len(arr)
        full_res[day] = res